In [19]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
import pickle
from sklearn.svm import SVC

In [20]:
#this function load the RGB image and change to gray inage, change the 2D image data to 1D and form a matrix
def load_image_to_vector(image_path,image_number):
    img_data=[]
    for i in range (image_number):
        img_color= mpimg.imread(image_path + str(i) + '.png')
        img_color=cv2.resize(img_color,(100,100))
        img=cv2.cvtColor(img_color,cv2.COLOR_BGR2GRAY)
        img_vector=img.reshape(10000)
        img_data.append(img_vector)
    return img_data

In [21]:
#PCA is used for dimensionality reduction
def img_data_pca(img_data,dimention):
    pca = PCA(n_components = 100)
    pca.fit(img_data)
    pca_data=pca.transform(img_data)
    return pca_data

In [22]:
def get_label(label_path,label_name):
    label=pd.read_table(label_path)
    y=label[label_name]
    return y

In [23]:
def SVM_model(x_train,y_train,x_test,y_test):
    clf=SVC(gamma='auto')
    clf.fit(x_train,y_train)
    y_pred =  clf.predict(x_test)
    print('Accuracy on train set:'+str(clf.score(x_train,y_train)))
    print('Accuracy on test set: '+str(accuracy_score(y_test,y_pred)))
    print(classification_report(y_test,y_pred))
    return clf

In [25]:
#get image data
img_data_train=load_image_to_vector('../datasets/cartoon_set/img/',5000)
img_data_test=load_image_to_vector('../datasets/cartoon_set_test/img/',2500)
#pca for image data
x_train=img_data_pca(img_data_train,100)
x_test=img_data_pca(img_data_test,100)
#get label
y_train=get_label('../datasets/cartoon_set/labels.csv','face_shape')
y_train=y_train[:5000]
y_test=get_label('../datasets/cartoon_set_test/labels.csv','face_shape')
#Standardize the data
x_train = StandardScaler().fit_transform(x_train)
x_test = StandardScaler().fit_transform(x_test)
#train the model and report accuracy
model=SVM_model(x_train,y_train,x_test,y_test)
#save model
pickle.dump(model,open("SVM_shape.dat","wb")) 

Accuracy on train set:0.9966
Accuracy on test set: 0.3456
              precision    recall  f1-score   support

           0       0.15      0.18      0.16       500
           1       0.26      0.25      0.26       500
           2       0.94      0.78      0.85       500
           3       0.14      0.14      0.14       500
           4       0.36      0.38      0.37       500

    accuracy                           0.35      2500
   macro avg       0.37      0.35      0.36      2500
weighted avg       0.37      0.35      0.36      2500

